In [122]:
import requests
import bs4 as bs
import lxml

httpaddr = "https://www.clarin.com"

requests_ret = requests.get(httpaddr)
print(requests_ret, end = ' || ')
soup = bs.BeautifulSoup(requests_ret.text, 'lxml')
# Notas:
# Everything semmantically a header, is tagged with <h2>. To all scrapping practical
# purposes, all headers except h2 are deemded irrelevant
articles = soup.find_all("article"); print("articles :", len(articles), end = ' / ')
sections = soup.find_all("section"); print("sections :", len(sections), end = ' / ')
headers = soup.find_all("header"); print("headers :", len(headers), end = ' / ')
h1s = soup.find_all("h1"); print("h1s :", len(h1s), end = ' / ')
h2s = soup.find_all("h2"); print("h2s :", len(h2s), end = ' / ')
h3s = soup.find_all("h3"); print("h3s :", len(h3s), end = ' / ')
h4s = soup.find_all("h4"); print("h4s :", len(h4s), end = ' / ')
h5s = soup.find_all("h5"); print("h5s :", len(h5s), end = ' / ')
h6s = soup.find_all("h6"); print("h6s :", len(h6s))
# only <h2>s hold relevance
# the only <h1> holds an SVG graphic

for discard_tag in ("script", "style"):
    for t in soup.find_all(discard_tag): t.extract()

<Response [200]> || articles : 103 / sections : 18 / headers : 15 / h1s : 1 / h2s : 138 / h3s : 0 / h4s : 0 / h5s : 0 / h6s : 0


In [123]:
# DUMPS A REPORT ON THE NESTING OF <article>s IN <section>s:
# TODO: This can be generalized as a routine that, for any two lists of nodes, gives a report on nesting

# Check for section nesting
# There's an almost strict hierarchical nesting in which articles are always inside some section
# The double belonging of to sections 2 and 3 are due to the fact that section 2 is nested inside section 3 (July 24, 2022)
print("+++ <section> NESTING CHECK :: ")
for (i, j) in [(i, j) for (i, section_i) in enumerate(sections) for (j, section_j) in enumerate(sections) if section_j in section_i.descendants]:
    print(("\t<section>[%" + str(len(str(len(sections)))) + "d] is nested in <section>[%" + str(len(str(len(sections)))) + "d]") % (j, i))

print("+++ <article> NESTING CHECK :: ")
for (i, j) in [(i, j) for (i, article_i) in enumerate(articles) for (j, article_j) in enumerate(articles) if article_j in article_i.descendants]:
    print(("\t<article>[%" + str(len(str(len(articles)))) + "d] is nested in <article>[%" + str(len(str(len(articles)))) + "d]") % (j, i))

# dangling <article>s
# these, the <article>s not-in-any-section, are the articles depicted as "LO MAS VISTO POR LOS SUSCRIPTORES" at the tail of the page
articles_not_in_sections = articles[:]              # copy articles ...
articles_in_sections_set = set()
for section in sections:
    for article in articles:
        if article in section.descendants:
            articles_in_sections_set = articles_in_sections_set | {article} # need to use this notation. set() notation consumes an interable and therefore iterates over article's members
for article in articles_in_sections_set:
    while article in articles_not_in_sections:      # while instead of if, just in case of repetition
        articles_not_in_sections.remove(article)    # ... then substract every article already under a <section>
print("+++ NUMBER OF <article>s NOT UNDER ANY <section> :", len(articles_not_in_sections))

# Report which <article> is nested in which <section>s
maxlen = max( (len('section[' + 'x' * len(str(len(sections))) + ']:' + str(section.attrs['class'])) for section in sections) )
for i, section in enumerate(sections):
    line = (('section[%' + str(len(str(len(sections)))) + 'd]: %s' + ' ' * 100) % (i, str(section.attrs['class']),))[:maxlen] + " ||"
    for article in articles: line += "X" if article in section.descendants else '.'; 
    line += "|"
    # get nearest <header> previous sibling -- filter only tags
    cursor = section
    while cursor and cursor.name != 'header':
        cursor = cursor.previous_sibling
        try:
            if cursor.name == 'header':
                line += ' ' + cursor.text.strip()
        except:
            pass
        
    print(line)

+++ <section> NESTING CHECK :: 
	<section>[ 3] is nested in <section>[ 2]
+++ <article> NESTING CHECK :: 
+++ NUMBER OF <article>s NOT UNDER ANY <section> : 6
section[ 0]: ['container', 'no-p', 'listado']                              ||XXXXX..................................................................................................|
section[ 1]: ['container', 'no-p', 'listado']                              ||.....XXXXXXXXXXXXXX....................................................................................|
section[ 2]: ['container', 'listado', 'no-p']                              ||...................XXXXXXXXXXXX........................................................................|
section[ 3]: ['columnas']                                                  ||...................XXXXXX..............................................................................|
section[ 4]: ['container', 'listado', 'no-p']                              ||...............................XXXXX

In [124]:
articles_by_section = [[] for _ in sections]    # ... beware of reference replication...
for i, section in enumerate(sections):
    for j, article in enumerate(articles):
        if article in section.descendants: 
            articles_by_section[i].append(article)

# for i, art_by_sect in enumerate(articles_by_section): print(("<section>[%0" + str(len(str(len(sections)))) + "d]") % (i, ), "contains", len(art_by_sect), "articles")

articles_by_section_dict = {i : l for (i, l) in enumerate(articles_by_section)}
articles_by_section_dict.update({'none' : articles_not_in_sections})
# for k in articles_by_section_dict:
#     print(("For articles in <section>[%" + str(max([len(str(k)) for k in articles_by_section_dict])) + "s] number of articles:") % k, len(articles_by_section_dict[k]))

idx_width = max([len(str(k)) for k in articles_by_section_dict])
val_width = max([len(str(len(articles_by_section_dict[k]))) for k in articles_by_section_dict])
for i, k in enumerate(articles_by_section_dict):
    print(("<section>[%" + str(idx_width) + "s] " + "= %" + str(val_width) + "s") % 
          (k, str(len(articles_by_section_dict[k]))), end = '\n' if (i + 1) % 6 == 0 else ' // ')


<section>[   0] =  5 // <section>[   1] = 14 // <section>[   2] = 12 // <section>[   3] =  6 // <section>[   4] =  7 // <section>[   5] =  4
<section>[   6] =  5 // <section>[   7] =  4 // <section>[   8] =  6 // <section>[   9] =  6 // <section>[  10] =  6 // <section>[  11] =  6
<section>[  12] =  3 // <section>[  13] =  1 // <section>[  14] =  6 // <section>[  15] =  4 // <section>[  16] =  4 // <section>[  17] =  4
<section>[none] =  6 // 

In [125]:
# dumping some data by article as classified in articles_by_section_dict
for k, a_list in articles_by_section_dict.items():
    for j, article in enumerate(a_list):
        
        h2ss = article.find_all('h2') if article.find_all('h2') else None
        
        volanta = article.find(class_ = 'volanta') if article.find(class_ = 'volanta') else ''
        author = article.find(class_ = 'author') if article.find(class_ = 'author') else ''
        author_rel = article.find(class_ = 'author-rel') if article.find(class_ = 'author-rel') else ''
        summary = article.find(class_ = 'summary') if article.find(class_ = 'summary') else ''
        section = article.find(class_ = 'section') if article.find(class_ = 'section') else ''
        section_rel = article.find(class_ = 'section-rel') if article.find(class_ = 'section-rel') else ''
        
        print("<section>[%s] / <article>[%d]" % (str(k), j))
        print('-' * len("<section>[%s] / <article>[%d]" % (str(k), j)))
        if len(h2ss) != 1:
            if len(h2ss) > 1:
                print('\t++++++++MORE THAN ONE <h2>!!!++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
            else:
                print('\t++++++++NO <h2>!!!+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
        for i, h in enumerate(h2ss):
            print('\th2[%d] = %s' % (i, h))
#         print( '\tvolanta = "%s"' % (volanta, ),
#                '\n\tauthor = "%s"' % (author, ),
#                '\n\tsummary = "%s"' % (summary, ),
#                '\n\tsection = "%s"' % (section, ))
#         if volanta : print('\tvolanta = "%s"' % (volanta.strip()[:60], ))
#         if author : print('\tauthor = "%s"' % (author.strip()[:60], ))
#         if summary : print('\tsummary = "%s"' % (summary.strip()[:60], ))
#         if section : print('\tsection = "%s"' % (section.strip()[:60], ))
        if volanta : print('\tvolanta = "%s"' % (str(volanta)[:80], ))
        if author : print('\tauthor = "%s"' % (str(author)[:80], ))
        if author_rel : print('\tauthor = "%s"' % (str(author_rel)[:80], ))
        if summary : print('\tsummary = "%s"' % (str(summary)[:80], ))
        if section : print('\tsection = "%s"' % (str(section)[:80], ))
        if section_rel : print('\tsection_rel = "%s"' % (str(section_rel)[:80], ))
        print()
    print('-' * 60)

<section>[0] / <article>[0]
---------------------------
	h2[0] = <h2>Cómo es la fase 2 del plan de Massa: guiño al campo y suba de tasas para evitar la devaluación</h2>
	volanta = "<p class="volanta">Crisis económica</p>"
	author = "<p class="author">Juan Manuel Barca</p>"
	summary = "<p class="summary">El nuevo ministro avanza con medidas para frenar la pérdida d"

<section>[0] / <article>[1]
---------------------------
	h2[0] = <h2 class="title-rel">Quita de subsidios: cómo es la nueva segmentación que aplicará Massa en gas, luz y agua</h2>
	section_rel = "<span class="section-rel">Economía.</span>"

<section>[0] / <article>[2]
---------------------------
	h2[0] = <h2>Quién es y qué piensa Gabriel Rubinstein, el economista que será viceministro de Massa</h2>
	volanta = "<p class="volanta">Cambios en el Gobierno</p>"
	section = "<p class="section">Economía</p>"

<section>[0] / <article>[3]
---------------------------
	h2[0] = <h2>El dólar blue mantiene la tendencia y cotiza en alza</h

In [126]:
# dump source under every single <article>:
for i, article in enumerate(articles_by_section[0]):
    print("<article>[%d] : %s" % (i, len(article.parent.find_all('li', class_ = 'item-related'))))
    print("-------------")
    
    print(article.parent.prettify())
    print('-' * 60)

<article>[0] : 1
-------------
<li class="m-box child h300 col-lg-6 col-md-8 col-sm-12 col-xs-12 no-p m2x1" data-classes="m-box child m2x1 h300  col-lg-6 col-md-8 col-sm-12 col-xs-12 no-p" data-name="content" data-size-h="1" data-size-w="2">
 <article class="content-nota list-format twoxone_no_foto">
  <div class="mt">
   <div class="max-h flex-change">
    <h2>
     Cómo es la fase 2 del plan de Massa: guiño al campo y suba de tasas para evitar la devaluación
    </h2>
    <p class="volanta">
     Crisis económica
    </p>
   </div>
   <div class="data-txt">
    <span class="paywall-ico">
    </span>
    <p class="author">
     Juan Manuel Barca
    </p>
   </div>
   <p class="summary">
    El nuevo ministro avanza con medidas para frenar la pérdida de reservas, mientras completa su equipo. Ejes y claves de lo que viene.
   </p>
  </div>
  <a aria-label="/economia/plan-massa-fase-2-guino-campo-suba-tasas-evitar-devaluacion_0_VRDCCObQIV.html" class="link_article" href="/economia/plan-m

In [127]:
# How many <li>s are on each <article>?

for i, article in enumerate(articles):
    print("<article>[%3s]" % i, str(len(list(article.parent.find_all('li')))) if len(list(article.parent.find_all('li'))) != 0 else "", 
          end = ' ' if (i+1) % 8 != 0 else '\n')

<article>[  0] 1 <article>[  1]  <article>[  2]  <article>[  3]  <article>[  4]  <article>[  5]  <article>[  6]  <article>[  7] 
<article>[  8]  <article>[  9]  <article>[ 10]  <article>[ 11]  <article>[ 12]  <article>[ 13]  <article>[ 14]  <article>[ 15] 
<article>[ 16]  <article>[ 17]  <article>[ 18]  <article>[ 19]  <article>[ 20]  <article>[ 21]  <article>[ 22]  <article>[ 23] 
<article>[ 24]  <article>[ 25]  <article>[ 26]  <article>[ 27]  <article>[ 28]  <article>[ 29]  <article>[ 30]  <article>[ 31] 
<article>[ 32]  <article>[ 33]  <article>[ 34]  <article>[ 35]  <article>[ 36]  <article>[ 37]  <article>[ 38]  <article>[ 39] 
<article>[ 40]  <article>[ 41]  <article>[ 42]  <article>[ 43]  <article>[ 44]  <article>[ 45]  <article>[ 46]  <article>[ 47] 
<article>[ 48]  <article>[ 49]  <article>[ 50]  <article>[ 51]  <article>[ 52]  <article>[ 53]  <article>[ 54]  <article>[ 55] 
<article>[ 56]  <article>[ 57]  <article>[ 58]  <article>[ 59]  <article>[ 60]  <article>[ 61]  <articl

In [128]:
for i, art in enumerate(articles_by_section_dict[1][7:]):
    print(i, '\n', art.parent.parent.parent)

0 
 <div class="last-moment" id="lmvmod_202206596043799" xmlns="http://www.w3.org/1999/html">
<a class="tag" href="https://www.clarin.com/ultimo-momento/" style="cursor: pointer;" target="_blank"></a>
<ul class="box-items">
<li aria-label="/mundo/sequia-historica-deja-francia-agua-potable_0_3GmSbPYGgJ.html" onclick="window.open('/mundo/sequia-historica-deja-francia-agua-potable_0_3GmSbPYGgJ.html'); metricsView('bloques_home','bloque_ultimo_momento|3GmSbPYGgJ', true);" style="cursor: pointer;">
<article class="flex-change">
<h2>Una sequía histórica deja a Francia sin agua potable</h2>
<span>15:00</span>
</article>
</li>
<li aria-label="/deportes/-quiero-maradona-video-nino-argentino-llorando-pobreza-conmovio-club-brasileno_0_dRA1qqtpKW.html" onclick="window.open('/deportes/-quiero-maradona-video-nino-argentino-llorando-pobreza-conmovio-club-brasileno_0_dRA1qqtpKW.html'); metricsView('bloques_home','bloque_ultimo_momento|dRA1qqtpKW', true);" style="cursor: pointer;">
<article class="flex

In [129]:
# treewalk a given node and dump source structure under that node's tag:
def dumptree(n, prefix):
    # global articles
    # global sections   
    stack = []
    stack.extend(reversed([ch for ch in n.children if type(ch) == bs.element.Tag]))     # Warning! Generators tend to explode...
    print(prefix + n.name + ' // ', end = '')
    try:
        if n.name == 'h2' : print(n.attrs, '/', n.get_text().strip()[:1000])
        elif n.name == 'p' : print(n.attrs, '/', n.get_text().strip()[:100])
        elif n.name == 'span' : print(n.attrs, '/', n.get_text().strip()[:100])
        elif  n.name == 'a' : print(n.attrs['href'][:100], '/', ' '.join(n.get_text().strip().split('\n')))
        elif  n.name == 'div' : print(n.attrs['class'][:100])
        elif  n.name == 'section':
            print(n.name + '[' + str(sections.index(n)) + ']')
        elif  n.name == 'article':
            print(n.name + '[' + str(articles.index(n)) + ']')
        else:
            print(n.name)
    except KeyError:
        print("+++ ERROR WHILE TRYING TO ACCESS NONEXISTANT KEY!!!+++")
    while stack:
        dumptree(stack.pop(), '|   ' + prefix)
    return None


i = 0
print("<section>[%02d]" % i)
print("-------------")
dumptree(sections[i].parent, '')


<section>[00]
-------------
div // ['container', 'father', 'no-p']
|   div // ['contenedor-streaming']
|   |   div // +++ ERROR WHILE TRYING TO ACCESS NONEXISTANT KEY!!!+++
|   |   |   span // {'id': 'closeStreming'} / 
|   |   |   input // input
|   |   |   label // label
|   |   |   div // ['box-streaming']
|   |   |   |   div // ['live']
|   |   |   |   div // ['box-live-tx']
|   |   |   |   |   span // {'class': ['volanta']} / Juicio a Cristina Kirchner por la obra pública
|   |   |   |   |   h2 // {} / "Las licitaciones tenían nombre y apellido, todo fue una farsa", dijo el fiscal Luciani
|   |   |   |   |   |   a // https://www.clarin.com/politica/juicio-cristina-kirchner-vivo-tercer-dia-alegatos-delitos-obra-publi / "Las licitaciones tenían nombre y apellido, todo fue una farsa", dijo el fiscal Luciani
|   |   |   |   div // ['mg-fig']
|   |   |   |   |   picture // picture
|   |   |   |   |   |   iframe // iframe
|   |   div // ['box-streaming-off']
|   |   |   div // +++ ERROR

In [130]:
parent_lis = []
gparent_uls = []
ggparent_uls = []
gggparent_uls = []
ggggparent_uls = []

for article in articles:
    if article.parent.name == 'li':
        parent_lis.append(article.parent)
        gparent_uls.append(article.parent.parent)
        ggparent_uls.append(article.parent.parent.parent)
        gggparent_uls.append(article.parent.parent.parent.parent)
        ggggparent_uls.append(article.parent.parent.parent.parent.parent)

i = 5
for s in "parent_lis", "gparent_uls", "ggparent_uls", "gggparent_uls", "ggggparent_uls":
    exec(f'dumptree({s}[{i}], "")')
    print()

li // li
|   article // article[5]
|   |   div // ['mt']
|   |   |   div // ['flex-change']
|   |   |   |   h2 // {} / Confirman procesamientos por las entregas de plata negra en el departamento de CFK
|   |   |   |   p // {'class': ['volanta']} / Cuadernos de las coimas
|   |   |   div // ['data-txt']
|   |   |   |   p // {'class': ['author']} / Daniel Santoro
|   |   div // ['wrap-figure']
|   |   |   picture // picture
|   |   |   |   source // source
|   |   |   |   img // img
|   |   a // /politica/cuadernos-coimas-ii-camara-federal-confirmo-procesamientos-roberto-baratta-ex-funcionarios / 

ul // ul
|   li // li
|   |   article // article[5]
|   |   |   div // ['mt']
|   |   |   |   div // ['flex-change']
|   |   |   |   |   h2 // {} / Confirman procesamientos por las entregas de plata negra en el departamento de CFK
|   |   |   |   |   p // {'class': ['volanta']} / Cuadernos de las coimas
|   |   |   |   div // ['data-txt']
|   |   |   |   |   p // {'class': ['author']} / Daniel

In [131]:
# def dumptreeul(n, prefix):
def dumptreeul(n, prefix):
    tags_to_show = {'h2', 'div', 'ul', 'section', 'article', 'header', 'span', 'p', 'li'}
    
    stack = []
    stack.extend(reversed([ch for ch in n.children if type(ch) == bs.element.Tag]))

    if n.name in tags_to_show:
        print(prefix + n.name, end = '')
        if n in sections:
            print(f"[{sections.index(n)}]:", end = '')
        if n in articles:
            print(f"[{articles.index(n)}]:", end = '')
        if n in headers:
            print(f"[{headers.index(n)}]:", end = '')
        try:
            print('', n.attrs, end = '')
        except:
            print("+++ NO ATTRIBUTES +++", end = '')
        print()
        
        try:
            if n.name in {'h2', 'p', 'header'}:
                print(prefix + '\t', "<CRLF>".join(n.text.split('\n')))
        except:
            print(prefix + '\t')

    while stack:
            dumptreeul(stack.pop(), '| ' + prefix)
    return None


In [132]:
# BY <section> -- <section>[0] -- MAIN HEADLINES
dumptreeul(sections[0].parent, '')

div {'class': ['container', 'father', 'no-p'], 'data-classes': 'container', 'data-type': 'container_father', 'data-name': 'container_father'}
| div {'class': ['contenedor-streaming']}
| | div {'id': 'content-streaming'}
| | | span {'id': 'closeStreming'}
| | | div {'class': ['box-streaming'], 'id': 'box-streaming', 'data-size-w': '0', 'data-size-h': '1'}
| | | | div {'class': ['live']}
| | | | div {'class': ['box-live-tx']}
| | | | | span {'class': ['volanta']}
| | | | | h2 {}
| | | | | 	 "Las licitaciones tenían nombre y apellido, todo fue una farsa", dijo el fiscal Luciani
| | | | div {'id': 'streamingBox_mod_202275410334339', 'class': ['mg-fig']}
| | div {'id': 'streaming-off', 'class': ['box-streaming-off'], 'data-ref': 'https://www.youtube.com/embed/4M5Qb_bc89E?autoplay=true&mute=true', 'data-title': '', 'las': '', 'licitaciones': '', 'ten': '', 'nombre': '', 'y': '', 'apellido': '', 'todo': '', 'fue': '', 'una': '', 'farsa': '', 'dijo': '', 'el': '', 'fiscal': '', 'luciani': ''}


In [133]:
# BY <section> -- <section>[1] -- Includes last moment. Sibling with main headline
dumptreeul(sections[1].parent, '')

div {'class': ['container', 'father', 'no-p'], 'data-classes': 'container', 'data-type': 'container_father', 'data-name': 'container_father'}
| div {'class': ['contenedor-streaming']}
| | div {'id': 'content-streaming'}
| | | span {'id': 'closeStreming'}
| | | div {'class': ['box-streaming'], 'id': 'box-streaming', 'data-size-w': '0', 'data-size-h': '1'}
| | | | div {'class': ['live']}
| | | | div {'class': ['box-live-tx']}
| | | | | span {'class': ['volanta']}
| | | | | h2 {}
| | | | | 	 "Las licitaciones tenían nombre y apellido, todo fue una farsa", dijo el fiscal Luciani
| | | | div {'id': 'streamingBox_mod_202275410334339', 'class': ['mg-fig']}
| | div {'id': 'streaming-off', 'class': ['box-streaming-off'], 'data-ref': 'https://www.youtube.com/embed/4M5Qb_bc89E?autoplay=true&mute=true', 'data-title': '', 'las': '', 'licitaciones': '', 'ten': '', 'nombre': '', 'y': '', 'apellido': '', 'todo': '', 'fue': '', 'una': '', 'farsa': '', 'dijo': '', 'el': '', 'fiscal': '', 'luciani': ''}


In [134]:
# BY <section> -- <section>[2] -- "Las Firmas" -- Commentary & Opinion Articles
dumptreeul(sections[2].parent, '')

div {'class': ['container', 'father', 'no-p'], 'data-classes': 'container', 'data-type': 'container_father', 'data-name': 'container_father'}
| section[2]: {'data-name': 'cont3B', 'class': ['container', 'listado', 'no-p'], 'data-classes': 'container listado ', 'data-type': 'container'}
| | ul {'class': ['count-first', 'bb-container', 'three-floor-banner', 'bb-moduleId_3B_mod_202164929140454'], 'data-count': '12', 'data-classes': ''}
| | | li {'class': ['m-box', 'gray_bg', 'h600', 'col-lg-6', 'col-md-8', 'col-sm-12', 'col-xs-12', 'no-p', 'size2x2'], 'data-classes': 'm-box h600 col-lg-6 col-md-8 col-sm-12 col-xs-12 no-p size2x2', 'data-size-w': '2', 'data-size-h': '2', 'data-name': 'content'}
| | | | section[3]: {'class': ['columnas'], 'id': 'columnas'}
| | | | | div {'class': ['title']}
| | | | | ul {'class': ['main__container']}
| | | | | | li {}
| | | | | | | article[19]: {'class': ['colum__container', 'small__colum']}
| | | | | | | | div {'class': ['mt', 'small__content']}
| | | | | 

In [135]:
# BY <section> -- <section>[3]
dumptreeul(sections[3].parent, '')

li {'class': ['m-box', 'gray_bg', 'h600', 'col-lg-6', 'col-md-8', 'col-sm-12', 'col-xs-12', 'no-p', 'size2x2'], 'data-classes': 'm-box h600 col-lg-6 col-md-8 col-sm-12 col-xs-12 no-p size2x2', 'data-size-w': '2', 'data-size-h': '2', 'data-name': 'content'}
| section[3]: {'class': ['columnas'], 'id': 'columnas'}
| | div {'class': ['title']}
| | ul {'class': ['main__container']}
| | | li {}
| | | | article[19]: {'class': ['colum__container', 'small__colum']}
| | | | | div {'class': ['mt', 'small__content']}
| | | | | | span {'class': ['author-name', 'small__name']}
| | | | | | h2 {'class': ['small__h2']}
| | | | | | 	 El acuerdo de Cristina con Massa se verá en las cuentas bonaerenses
| | | li {}
| | | | article[20]: {'class': ['colum__container', 'small__colum']}
| | | | | div {'class': ['mt', 'small__content']}
| | | | | | span {'class': ['author-name', 'small__name']}
| | | | | | h2 {'class': ['small__h2']}
| | | | | | 	 Después de los anuncios, ahora "la moneda está en el aire"
| | |

In [136]:
# BY <section> -- <section>[4]
dumptreeul(sections[4].parent, '')

div {'class': ['container', 'father', 'no-p'], 'data-classes': 'container', 'data-type': 'container_father', 'data-name': 'container_father'}
| section[4]: {'data-name': 'cont3B', 'class': ['container', 'listado', 'no-p'], 'data-classes': 'container listado ', 'data-type': 'container'}
| | ul {'class': ['count-first', 'bb-container', 'three-floor-banner', 'bb-moduleId_3B_mod_202225241679411'], 'data-count': '12', 'data-classes': ''}
| | | li {'class': ['suscribers-only', 'm-box', 'child', 'h600', 'col-lg-6', 'col-md-8', 'col-sm-12', 'col-xs-12', 'no-p', 'size2x2', 't-top', 't-right'], 'data-size-w': '2', 'data-size-h': '2', 'data-classes': 'suscribers-only child m-box h600 col-lg-6 col-md-8 col-sm-12 col-xs-12 size2x2 t-top t-right', 'data-name': 'content'}
| | | | article[31]: {'class': ['visual', 'bottom_degrade_large', 'content-nota', 'list-format', 'twoxtwo_visual', 't-top', 't-right']}
| | | | | div {'class': ['mt', 'title-bottom']}
| | | | | | div {'class': ['max-h']}
| | | | | |

In [137]:
# BY <section> -- <section>[5]
print(
    [sec.name for sec in sections[5].previous_siblings]
)

dumptreeul(sections[5].parent, '')

[None, 'header', None]
div {'class': ['carcasaSkin']}
| header[2]: {'class': ['full', 'item-content', 'orange-section', 'back'], 'data-classes': 'full item-content orange-section back', 'data-name': 'separador', 'data-linkcontent': 'https://www.clarin.com/fama/'}
| 	 <CRLF><CRLF><CRLF><CRLF>FAMA<CRLF><CRLF><CRLF>
| | div {'class': ['separador-home']}
| | | div {'class': ['content-separador']}
| | | | h2 {'class': ['heading']}
| | | | 	 FAMA
| section[5]: {'data-name': 'cont1A', 'class': ['container', 'no-p', 'listado'], 'data-classes': 'container no-p listado ', 'data-type': 'container'}
| | ul {'class': ['count-first', 'bb-container', 'one-floor-A', 'bb-moduleId_1A_mod_202164244066916', 'orange-section', 'back'], 'data-classes': ' orange-section back', 'data-count': '4'}
| | | li {'class': ['m-box', 'h300', 'child', 'col-lg-3', 'col-md-4', 'col-sm-6', 'col-xs-12', 'no-p', 'list-format'], 'data-size-w': '1', 'data-size-h': '1', 'data-classes': 'm-box child h300 col-lg-3 col-md-4 col-sm

In [138]:
# BY <section> -- <section>[6]
dumptreeul(sections[6].parent, '')

div {'class': ['carcasaSkin']}
| header[3]: {'class': ['full', 'item-content', 'suscriptor-content', 'back'], 'data-classes': 'full item-content suscriptor-content back', 'data-name': 'separador', 'data-linkcontent': ''}
| 	 <CRLF><CRLF><CRLF><CRLF>Exclusivo suscriptores<CRLF><CRLF><CRLF>
| | div {'class': ['separador-home']}
| | | div {'class': ['content-separador']}
| | | | h2 {'class': ['heading']}
| | | | 	 Exclusivo suscriptores
| section[6]: {'data-name': 'cont2B', 'class': ['container', 'no-p', 'listado', 'suscriptor-content', 'back'], 'data-classes': 'container listado suscriptor-content back', 'data-type': 'container'}
| | ul {'class': ['count-first', 'bb-container', 'two-floor', 'bb-moduleId_2B_mod_2021112110957595', 'suscriptor-content', 'back'], 'data-classes': 'suscriptor-content back', 'data-count': '8'}
| | | li {'class': ['suscribers-only', 'm-box', 'child', 'h600', 'col-lg-6', 'col-md-8', 'col-sm-12', 'col-xs-12', 'no-p', 'size2x2', 't-bottom', 't-left'], 'data-size-w'

In [139]:
# BY <section> -- <section>[7]
dumptreeul(sections[0].parent, '')

div {'class': ['container', 'father', 'no-p'], 'data-classes': 'container', 'data-type': 'container_father', 'data-name': 'container_father'}
| div {'class': ['contenedor-streaming']}
| | div {'id': 'content-streaming'}
| | | span {'id': 'closeStreming'}
| | | div {'class': ['box-streaming'], 'id': 'box-streaming', 'data-size-w': '0', 'data-size-h': '1'}
| | | | div {'class': ['live']}
| | | | div {'class': ['box-live-tx']}
| | | | | span {'class': ['volanta']}
| | | | | h2 {}
| | | | | 	 "Las licitaciones tenían nombre y apellido, todo fue una farsa", dijo el fiscal Luciani
| | | | div {'id': 'streamingBox_mod_202275410334339', 'class': ['mg-fig']}
| | div {'id': 'streaming-off', 'class': ['box-streaming-off'], 'data-ref': 'https://www.youtube.com/embed/4M5Qb_bc89E?autoplay=true&mute=true', 'data-title': '', 'las': '', 'licitaciones': '', 'ten': '', 'nombre': '', 'y': '', 'apellido': '', 'todo': '', 'fue': '', 'una': '', 'farsa': '', 'dijo': '', 'el': '', 'fiscal': '', 'luciani': ''}


In [140]:
# BY <section> -- <section>[8]
dumptreeul(sections[8].parent, '')

div {'class': ['container', 'father', 'no-p'], 'data-classes': 'container', 'data-type': 'container_father', 'data-name': 'container_father'}
| div {'class': ['carcasaSkin']}
| | header[2]: {'class': ['full', 'item-content', 'orange-section', 'back'], 'data-classes': 'full item-content orange-section back', 'data-name': 'separador', 'data-linkcontent': 'https://www.clarin.com/fama/'}
| | 	 <CRLF><CRLF><CRLF><CRLF>FAMA<CRLF><CRLF><CRLF>
| | | div {'class': ['separador-home']}
| | | | div {'class': ['content-separador']}
| | | | | h2 {'class': ['heading']}
| | | | | 	 FAMA
| | section[5]: {'data-name': 'cont1A', 'class': ['container', 'no-p', 'listado'], 'data-classes': 'container no-p listado ', 'data-type': 'container'}
| | | ul {'class': ['count-first', 'bb-container', 'one-floor-A', 'bb-moduleId_1A_mod_202164244066916', 'orange-section', 'back'], 'data-classes': ' orange-section back', 'data-count': '4'}
| | | | li {'class': ['m-box', 'h300', 'child', 'col-lg-3', 'col-md-4', 'col-sm-

In [141]:
# BY <section> -- <section>[9]
dumptreeul(sections[9].parent, '')

div {'class': ['carcasaSkin']}
| header[6]: {'class': ['full', 'item-content', 'grey-section', 'back'], 'data-classes': 'full item-content grey-section back', 'data-name': 'separador', 'data-linkcontent': 'https://www.clarin.com/economia/'}
| 	 <CRLF><CRLF><CRLF><CRLF>Economía y Servicios<CRLF><CRLF><CRLF>
| | div {'class': ['separador-home']}
| | | div {'class': ['content-separador']}
| | | | h2 {'class': ['heading']}
| | | | 	 Economía y Servicios
| section[9]: {'data-name': 'cont2WB', 'class': ['container', 'no-p', 'listado', 'grey-section', 'back'], 'data-classes': 'container listado grey-section back', 'data-type': 'container'}
| | ul {'class': ['bb-container', 'two-floor-banner', 'bb-moduleId_2WB_mod_202173729665159', 'grey-section', 'back'], 'data-count': '8', 'data-classes': 'grey-section back'}
| | | li {'class': ['border', 'box-large', 'col-lg-6', 'col-md-8', 'col-sm-12', 'col-xs-12', 'no-p', 'size2x2', 'twox_container', 'c-pad', 'special'], 'data-size-w': '2', 'data-size-h':

In [142]:
# BY <section> -- <section>[10]
dumptreeul(sections[10].parent, '')

div {'class': ['carcasaSkin']}
| header[7]: {'class': ['full', 'item-content'], 'data-classes': 'full item-content ', 'data-name': 'separador', 'data-linkcontent': ''}
| 	 <CRLF><CRLF><CRLF><CRLF>Estilo<CRLF><CRLF><CRLF>
| | div {'class': ['separador-home']}
| | | div {'class': ['content-separador']}
| | | | h2 {'class': ['heading']}
| | | | 	 Estilo
| section[10]: {'data-name': 'cont2WB', 'class': ['container', 'no-p', 'listado'], 'data-classes': 'container listado ', 'data-type': 'container'}
| | ul {'class': ['bb-container', 'two-floor-banner', 'bb-moduleId_2WB_mod_201756636671203'], 'data-count': '8', 'data-classes': ''}
| | | li {'class': ['border', 'box-large', 'col-lg-6', 'col-md-8', 'col-sm-12', 'col-xs-12', 'no-p', 'size2x2', 'twox_container', 'c-pad', 'special'], 'data-size-w': '2', 'data-size-h': '2', 'data-classes': 'border box-large col-lg-6 col-md-8 col-sm-12 col-xs-12 no-p size2x2 twox_container  special', 'data-name': 'container2x2'}
| | | | div {'class': ['top-head']}


In [143]:
# BY <section> -- <section>[11]
dumptreeul(sections[11].parent, '')

div {'class': ['carcasaSkin']}
| header[8]: {'class': ['full', 'item-content'], 'data-classes': 'full item-content ', 'data-name': 'separador', 'data-linkcontent': ''}
| 	 <CRLF><CRLF><CRLF><CRLF>Clic<CRLF><CRLF><CRLF>
| | div {'class': ['separador-home']}
| | | div {'class': ['content-separador']}
| | | | h2 {'class': ['heading']}
| | | | 	 Clic
| section[11]: {'data-name': 'cont2WB', 'class': ['container', 'no-p', 'listado'], 'data-classes': 'container listado ', 'data-type': 'container'}
| | ul {'class': ['bb-container', 'two-floor-banner', 'bb-moduleId_2WB_mod_202164463708654'], 'data-count': '8', 'data-classes': ''}
| | | li {'class': ['border', 'box-large', 'col-lg-6', 'col-md-8', 'col-sm-12', 'col-xs-12', 'no-p', 'size2x2', 'twox_container', 'c-pad', 'special'], 'data-size-w': '2', 'data-size-h': '2', 'data-classes': 'border box-large col-lg-6 col-md-8 col-sm-12 col-xs-12 no-p size2x2 twox_container  special', 'data-name': 'container2x2'}
| | | | div {'class': ['top-head']}
| | 

In [144]:
# BY <section> -- <section>[12]
dumptreeul(sections[12].parent, '')

div {'class': ['carcasaSkin']}
| header[9]: {'class': ['full', 'item-content'], 'data-classes': 'full item-content ', 'data-name': 'separador', 'data-linkcontent': 'https://www.youtube.com/c/clarincom'}
| 	 <CRLF><CRLF><CRLF><CRLF>Video <CRLF><CRLF><CRLF>
| | div {'class': ['separador-home']}
| | | div {'class': ['content-separador']}
| | | | h2 {'class': ['heading']}
| | | | 	 Video 
| section[12]: {'data-name': 'cont1C', 'class': ['container', 'no-p', 'listado'], 'data-classes': 'container no-p listado ', 'data-type': 'container'}
| | ul {'class': ['count-first', 'bb-container', 'one-floor-C', 'bb-moduleId_1C_mod_201754205720610'], 'data-classes': ' ', 'data-count': '4'}
| | | li {'class': ['m-box', 'child', 'h300', 'col-lg-6', 'col-md-8', 'col-sm-12', 'col-xs-12', 'no-p', 'm2x1', 't-top', 't-right'], 'data-size-w': '2', 'data-size-h': '1', 'data-classes': 'm-box child h300  col-lg-6 col-md-8 col-sm-12 col-xs-12 no-p t-top t-right', 'data-name': 'content'}
| | | | article[75]: {'clas

In [145]:
# BY <section> -- <section>[13]
dumptreeul(sections[13].parent, '')

div {'class': ['container', 'father', 'no-p'], 'data-classes': 'container', 'data-type': 'container_father', 'data-name': 'container_father'}
| div {'class': ['carcasaSkin']}
| | header[8]: {'class': ['full', 'item-content'], 'data-classes': 'full item-content ', 'data-name': 'separador', 'data-linkcontent': ''}
| | 	 <CRLF><CRLF><CRLF><CRLF>Clic<CRLF><CRLF><CRLF>
| | | div {'class': ['separador-home']}
| | | | div {'class': ['content-separador']}
| | | | | h2 {'class': ['heading']}
| | | | | 	 Clic
| | section[11]: {'data-name': 'cont2WB', 'class': ['container', 'no-p', 'listado'], 'data-classes': 'container listado ', 'data-type': 'container'}
| | | ul {'class': ['bb-container', 'two-floor-banner', 'bb-moduleId_2WB_mod_202164463708654'], 'data-count': '8', 'data-classes': ''}
| | | | li {'class': ['border', 'box-large', 'col-lg-6', 'col-md-8', 'col-sm-12', 'col-xs-12', 'no-p', 'size2x2', 'twox_container', 'c-pad', 'special'], 'data-size-w': '2', 'data-size-h': '2', 'data-classes': 'b

In [146]:
# BY <section> -- <section>[14]
dumptreeul(sections[14].parent, '')

div {'class': ['carcasaSkin']}
| header[11]: {'class': ['full', 'item-content'], 'data-classes': 'full item-content ', 'data-name': 'separador', 'data-linkcontent': 'https://www.clarin.com/brandstudio/'}
| 	 <CRLF><CRLF><CRLF><CRLF> BRAND STUDIO<CRLF><CRLF><CRLF>
| | div {'class': ['separador-home']}
| | | div {'class': ['content-separador']}
| | | | h2 {'class': ['heading']}
| | | | 	  BRAND STUDIO
| section[14]: {'data-name': 'cont2WB', 'class': ['container', 'no-p', 'listado'], 'data-classes': 'container listado ', 'data-type': 'container'}
| | ul {'class': ['bb-container', 'two-floor-banner', 'bb-moduleId_2WB_mod_202212722620787'], 'data-count': '8', 'data-classes': ''}
| | | li {'class': ['border', 'box-large', 'col-lg-6', 'col-md-8', 'col-sm-12', 'col-xs-12', 'no-p', 'size2x2', 'twox_container', 'c-pad', 'special'], 'data-size-w': '2', 'data-size-h': '2', 'data-classes': 'border box-large col-lg-6 col-md-8 col-sm-12 col-xs-12 no-p size2x2 twox_container  special', 'data-name': 'c

In [147]:
# BY <section> -- <section>[15]
dumptreeul(sections[15].parent, '')

div {'class': ['carcasaSkin']}
| header[12]: {'class': ['full', 'item-content'], 'data-classes': 'full item-content ', 'data-name': 'separador', 'data-linkcontent': ''}
| 	 <CRLF><CRLF><CRLF><CRLF>The New York Times International Weekly<CRLF><CRLF><CRLF>
| | div {'class': ['separador-home']}
| | | div {'class': ['content-separador']}
| | | | h2 {'class': ['heading']}
| | | | 	 The New York Times International Weekly
| section[15]: {'data-name': 'cont1B', 'class': ['container', 'no-p', 'listado'], 'data-classes': 'container listado', 'data-type': 'container'}
| | ul {'class': ['count-first', 'bb-container', 'one-floor', 'bb-moduleId_1B_mod_201881263003819'], 'data-count': '4', 'data-classes': ''}
| | | li {'class': ['m-box', 'child', 'h300', 'col-lg-6', 'col-md-8', 'col-sm-12', 'col-xs-12', 'no-p', 'm2x1', 't-bottom', 't-left'], 'data-size-w': '2', 'data-size-h': '1', 'data-classes': 'm-box child h300  col-lg-6 col-md-8 col-sm-12 col-xs-12 no-p t-bottom t-left', 'data-name': 'content'}


In [148]:
# BY <section> -- <section>[16]
dumptreeul(sections[16].parent, '')

div {'class': ['carcasaSkin']}
| header[13]: {'class': ['full', 'item-content'], 'data-classes': 'full item-content ', 'data-name': 'separador', 'data-linkcontent': 'https://www.clarin.com/zonales/'}
| 	 <CRLF><CRLF><CRLF><CRLF>Conurbano<CRLF><CRLF><CRLF>
| | div {'class': ['separador-home']}
| | | div {'class': ['content-separador']}
| | | | h2 {'class': ['heading']}
| | | | 	 Conurbano
| section[16]: {'data-name': 'cont1A', 'class': ['container', 'no-p', 'listado'], 'data-classes': 'container no-p listado ', 'data-type': 'container'}
| | ul {'class': ['count-first', 'bb-container', 'one-floor-A', 'bb-moduleId_1A_mod_202165832031849'], 'data-classes': ' ', 'data-count': '4'}
| | | li {'class': ['m-box', 'h300', 'child', 'col-lg-3', 'col-md-4', 'col-sm-6', 'col-xs-12', 'no-p', 'list-format'], 'data-size-w': '1', 'data-size-h': '1', 'data-classes': 'm-box child h300 col-lg-3 col-md-4 col-sm-6 col-xs-12 no-p list-format', 'data-name': 'content'}
| | | | article[89]: {'class': ['content-n

In [149]:
# BY <section> -- <section>[17]
dumptreeul(sections[17].parent, '')

div {'class': ['carcasaSkin']}
| header[14]: {'class': ['full', 'item-content'], 'data-classes': 'full item-content ', 'data-name': 'separador', 'data-linkcontent': 'https://elle.clarin.com/'}
| 	 <CRLF><CRLF><CRLF><CRLF>Elle<CRLF><CRLF><CRLF>
| | div {'class': ['separador-home']}
| | | div {'class': ['content-separador']}
| | | | h2 {'class': ['heading']}
| | | | 	 Elle
| section[17]: {'data-name': 'cont1B', 'class': ['container', 'no-p'], 'data-classes': 'container  no-p', 'data-type': 'container'}
| | ul {'class': ['count-first', 'bb-container', 'one-floor'], 'data-count': '4'}
| | | li {'class': ['m-box', 'child', 'h300', 'col-lg-6', 'col-md-8', 'col-sm-12', 'col-xs-12', 'no-p', 'm2x1'], 'data-size-w': '2', 'data-size-h': '1', 'data-classes': 'm-box child h300 col-lg-6 col-md-8 col-sm-12 col-xs-12 no-p m2x1', 'data-name': 'content'}
| | | | article[93]: {'class': ['visual', 'content-nota', 'twoxone_visual', 'bottom_degrade']}
| | | | | div {'class': ['mt']}
| | | | | | div {'class'

In [150]:
# BY <section> -- <section>[18]
dumptreeul(sections[15].parent, '')

div {'class': ['carcasaSkin']}
| header[12]: {'class': ['full', 'item-content'], 'data-classes': 'full item-content ', 'data-name': 'separador', 'data-linkcontent': ''}
| 	 <CRLF><CRLF><CRLF><CRLF>The New York Times International Weekly<CRLF><CRLF><CRLF>
| | div {'class': ['separador-home']}
| | | div {'class': ['content-separador']}
| | | | h2 {'class': ['heading']}
| | | | 	 The New York Times International Weekly
| section[15]: {'data-name': 'cont1B', 'class': ['container', 'no-p', 'listado'], 'data-classes': 'container listado', 'data-type': 'container'}
| | ul {'class': ['count-first', 'bb-container', 'one-floor', 'bb-moduleId_1B_mod_201881263003819'], 'data-count': '4', 'data-classes': ''}
| | | li {'class': ['m-box', 'child', 'h300', 'col-lg-6', 'col-md-8', 'col-sm-12', 'col-xs-12', 'no-p', 'm2x1', 't-bottom', 't-left'], 'data-size-w': '2', 'data-size-h': '1', 'data-classes': 'm-box child h300  col-lg-6 col-md-8 col-sm-12 col-xs-12 no-p t-bottom t-left', 'data-name': 'content'}


In [151]:
for i, section in enumerate(sections):
    for j, header in enumerate(headers):
        if header in section.previous_siblings:
            print('<header>', j, 'heads <section>', i, 'with text', header.text.strip())

<header> 2 heads <section> 5 with text FAMA
<header> 3 heads <section> 6 with text Exclusivo suscriptores
<header> 4 heads <section> 7 with text ESPECIALES CLARÍN
<header> 5 heads <section> 8 with text Deportes
<header> 6 heads <section> 9 with text Economía y Servicios
<header> 7 heads <section> 10 with text Estilo
<header> 8 heads <section> 11 with text Clic
<header> 9 heads <section> 12 with text Video
<header> 10 heads <section> 13 with text FOTOGALERIA
<header> 11 heads <section> 14 with text BRAND STUDIO
<header> 12 heads <section> 15 with text The New York Times International Weekly
<header> 13 heads <section> 16 with text Conurbano
<header> 14 heads <section> 17 with text Elle


<head>
<title>Noticias. Últimas noticias de Argentina y el Mundo | Clarín</title>

<link as="font" crossorigin="" href="https://www.clarin.com/static/CLAClarin/fonts-v2/VelinoSans-Bold.woff2" rel="preload" type="font/woff2"/>
<link as="font" crossorigin="" href="https://www.clarin.com/static/CLAClarin/fonts-v2/VelinoSans-Book.woff2" rel="preload" type="font/woff2"/>
<link as="font" crossorigin="" href="https://www.clarin.com/static/CLAClarin/fonts-v2/VelinoHeadline-Bold.woff2" rel="preload" type="font/woff2"/>
<link as="font" crossorigin="" href="https://www.clarin.com/static/CLAClarin/fonts-v2/VelinoHeadline-Book.woff2" rel="preload" type="font/woff2"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="es" http-equiv="Content-Language"/>
<meta content="es" name="LANGUAGE"/>
<meta content="Global" name="DISTRIBUTION"/>
<meta content="all" name="ROBOTS"/>
<meta content="Clarin.com" name="AUTHOR"/>
<meta content="noticias, información, videos, diario, ne